<a href="https://colab.research.google.com/github/Nadekoii/RumbaDesChiffres_IDAStar/blob/main/RumbaDesChiffres_IDAStar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implémentation Bibliothèques & des Structures de Données utilisés

### Les bibliothèques utilisées

In [ ]:
from posixpath import lexists
from dataclasses import dataclass
from typing import Any, List, Sequence
import math
import time

### Les Structure de Données utilisés:

#### Un cube: représenté simplement par un nombre

In [ ]:
@dataclass
class Cube:
  nom: int

#### Une tige: représenté par une liste LIFO (donc une pile) de taille 3

In [ ]:
@dataclass
class Tige:
  queue: list

  # Recuperer la taille de la pile
  def height(self):
    return(len(self.queue))

  def isEmpty(self):
    if self.height()==0:
      return True
    return False

  def isFull(self):
    if self.height()==3:
      return True
    return False

  # Ajouter/retirer des cubes
  def push(self,cube):
    if not self.isFull():
      self.queue.append(cube)
    else: print("ERREUR: tige plein !")

  def pop(self):
    if not self.isEmpty():
      return(self.queue.pop(self.height()-1))
    else: print("ERREUR: file vide !")

  # Retourner l'élément au sommet de la pile
  def top(self):
    if(not self.isEmpty()):
      return(self.queue[self.height()-1])
    else: print("ERREUR: file vide !")

  # Afficher la pile
  def print_tige(self):
    print(self.queue)

  # Copier un tige
  def tige_cp(self):
    tige_cp = Tige(self.queue.copy())
    return(tige_cp)

  def compare(self,t2):
    if self.height()!=t2.height():
      return False
    i=0
    while i<self.height():
      if self.queue[i].nom!=t2.queue[i].nom:
        return False
      i+=1
    return True

#### Un noeud: représente un état du jeu et qui est représenté par une liste de 4 tiges

In [ ]:
@dataclass
class Noeud:
  tiges: list = None

  # Initialiser les 4 tiges vides
  def init_noeud(self):
    i=0
    self.tiges = []
    while i<4:
      self.tiges.append(Tige([]))
      i+=1

  # Copier un noeud
  def noeud_cp(self):
    new_noeud = Noeud()
    new_noeud.tiges = []
    i=0
    while i<4:
      new_noeud.tiges.append(self.tiges[i].tige_cp())
      i+=1
    return new_noeud

  # Retourner le n-ième tige
  def get(self,n):
    return self.tiges[n]

  # Affichage du noeud
  def print_noeud(self):
    i = 3
    j = 0
    while i>0:
      print(i-1,"|  ",end='')
      while j<4:
        if (self.tiges[j].height()<i):
          print("  |  ",end='')
        else:
          print(" ",self.get(j).queue[i-1].nom," ",end='')
        j+=1
      j=0
      i-=1
      print()
    print("------------------------")
    print("       0    1    2    3")

  # Retourner la liste des tiges déplacable:
  def movable(self,a):
    i=0
    returned = []
    if not self.get(a).isEmpty():
      while i<4:
        if (i!=a)&(not self.tiges[i].isFull()):
          returned.append(i)
        i+=1
    return returned

  # Déplacer le sommet du a vers b
  def move(self,a,b):
    if b in self.movable(a):
      new_noeud = self.noeud_cp()
      element = new_noeud.get(a).queue.pop()
      new_noeud.get(b).queue.append(element)
      return new_noeud
    else: print("ERREUR: a est vide ou b est plein !")

  # Vérifier si l'état est le but cherché
  def isGoal(self,g):
    if self==g:
      return True
    False

  # Reset le noeud
  def clear(self):
    for tige in self.tiges:
      while not tige.isEmpty():
        tige.pop()

  # Comparer 2 noeuds (retourner True si =,sinon False)
  def compare(self,noeudComparer):
    i=0
    while i<4:
      if not (self.get(i).compare(noeudComparer.get(i))):
        return False
      i+=1
    return True

#### Un triplet: représente avec 3 valeurs operation, noeud fils et poids

In [ ]:
@dataclass
class Triplet:
  operation: tuple = None
  noeudFils: Noeud = None
  cout: int = None

### Autres fonctions:

In [ ]:
#Générer la liste des triplets depuis 1 état (operation, noeudFils, poids)
def opPos(noeud):
  listOps = []
  listTrip = []
  i=0
  while i<4:
    listOps = noeud.movable(i)
    for e in listOps:
      listTrip.append(Triplet((i,e),noeud.move(i,e)))
    i+=1
  return listTrip

In [ ]:
#Voir si une liste contenant des noeuds contient deja 1 noeud
def isInside(list_noeud,noeud):
  for n in list_noeud:
    if (n.compare(noeud)):
      return(True)
  return False

In [ ]:
#Printer le path
def print_path(path):
  for e in path:
    print(e.operation)
    e.noeudFils.print_noeud()

### Heuristiques:

In [ ]:
def nombreMalMis(depart,but):
  count = 0
  i = 0
  j = 0
  while i<4:
    while (j<depart.get(i).height()):
      if (but.get(i).height()-1<j)or(depart.get(i).queue[j].nom!=but.get(i).queue[j].nom):
        count+=1
      j+=1
    i+=1
  return count

In [ ]:
#Calculer le h() d'un état donc l'heuristique choisi
def h(depart,but):
  return nombreMalMis(depart,but)

### Implémentation IDA*

####IDA*

In [ ]:
def Ida_star(depart,but):
  solution = None
  seuil = h(depart,but)
  print(seuil)
  path = []
  termine = True
  num_ite = 0
  while termine:
    num_ite+=1
    termine,solution,seuil,path,num_noeuds_cree,num_noeuds_developpe = Profondeur_bornee(depart,but,seuil,path)
  if solution!=None:
    path.insert(0,Triplet((0,0),depart,0))
    return solution,path,num_ite,num_noeuds_cree,num_noeuds_developpe
  else:
    raise ValueError("No path has been found.")

In [ ]:
def Profondeur_bornee(depart,but,seuil,path):
  # Init Variables
  num_noeuds_cree = 1
  num_noeuds_developpe = 0
  nSeuil = math.inf
  vus = []
  file_attente = [Triplet((0,0),depart,0)]
  # Tant que file_attente pas vide
  while file_attente!=[]:
    # Prochain = pop(fileAttente)
    prochain = file_attente.pop(0)
    num_noeuds_developpe+=1
    # Ajouter prochain dans vus si il n'est pas deja dedans
    if not (isInside(vus,prochain.noeudFils)):
      vus.append(prochain.noeudFils)
    # Si prochain est l'etat but
    if prochain.noeudFils.isGoal(but):
      # On mettre a jour la solution
      solution = prochain.noeudFils.noeud_cp()
      path.append(Triplet(prochain.operation, prochain.noeudFils.noeud_cp(), prochain.cout))
      return False,solution,seuil,path,num_noeuds_cree,num_noeuds_developpe
    # Sinon
    else:
      # Pour tout triplet dans les etats Fils
      for trip in opPos(prochain.noeudFils):
        num_noeuds_cree+=1
        # fNoeud = g(noeud) + h(noeud) avec g=cout du prochain + h(noeud)
        trip.cout = 0
        f = h(trip.noeudFils,but) + trip.cout
        # si f de noeudFils <= seuil et pas dans la liste vus, on l'ajouter en tete de la liste
        if (f<=seuil) and (not isInside(vus,trip.noeudFils)):
          file_attente.insert(0,trip)
          path.append(prochain)
        # sinon on mettre a jour nSeuil
        else:
          nSeuil = min(nSeuil,f)
        # Fin si
      # Fin pour tout
    # Fin si
  # Fin tant que
  # Si nSeuil = infinite alors arreter
  if nSeuil==math.inf:
    return False,None,seuil,path,num_noeuds_cree,num_noeuds_developpe
  # Sinon on mettre a jours le seuil pour la prochaine iteration
  else:
    return True,None,nSeuil,[],num_noeuds_cree,num_noeuds_developpe

##Tests unitaires des SdDs

###Tests pour les cubes

In [ ]:
# Test d'initialisation du cube
cube = Cube(2)
cube

Cube(nom=2)

In [ ]:
# Fonction pour initialisé une liste de 9 cubes différents
def init_cubes():
  list_cubes = []
  for n in [1,2,3,4,5,6,7,8,9]:
    list_cubes.append(Cube(n))
  return list_cubes

In [ ]:
list_cubes = init_cubes()
print(list_cubes)

[Cube(nom=1), Cube(nom=2), Cube(nom=3), Cube(nom=4), Cube(nom=5), Cube(nom=6), Cube(nom=7), Cube(nom=8), Cube(nom=9)]


###Tests pour la création d'un noeud

In [ ]:
# Initialisation d'un noeud
noeud = Noeud([])
noeud.init_noeud()
print(noeud)

Noeud(tiges=[Tige(queue=[]), Tige(queue=[]), Tige(queue=[]), Tige(queue=[])])


In [ ]:
# Essai d'ajouter un cube dans l'état du jeu
noeud.get(0).push(list_cubes[0])
noeud.print_noeud()

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3


In [ ]:
# Essai de retirer le cube
popped_cube = noeud.tiges[0].pop()
noeud.print_noeud()
print(popped_cube)

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    |    |    |  
------------------------
       0    1    2    3
Cube(nom=1)


In [ ]:
# Essai de modifier tige vide
i=0
while(i<3):
  noeud.get(0).push(list_cubes[0])
  i+=1
i=0
while(i<4):
  noeud.get(0).pop()
  i+=1

ERREUR: file vide !


In [ ]:
# Essai de copier l'etat du jeu
noeud.print_noeud()
new_noeud = noeud.noeud_cp()
new_noeud.print_noeud()

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    |    |    |  
------------------------
       0    1    2    3


In [ ]:
# Essai de déplacer le cube
noeud.get(0).push(list_cubes[0])
new_noeud = noeud.move(0,1)
noeud.print_noeud()
new_noeud.print_noeud()

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    1    |    |  
------------------------
       0    1    2    3


In [ ]:
# Vider le noeud
noeud.clear()
noeud.print_noeud()

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    |    |    |  
------------------------
       0    1    2    3


###Tests pour les triplets

In [ ]:
# Initialisation d'un triplet
trip = Triplet((0,1),noeud,0)
print(trip)

Triplet(operation=(0, 1), noeudFils=Noeud(tiges=[Tige(queue=[]), Tige(queue=[]), Tige(queue=[]), Tige(queue=[])]), cout=0)


In [ ]:
# Tester OpPos
print(noeud)
noeud.get(0).push(list_cubes[0])
print(opPos(noeud))

Noeud(tiges=[Tige(queue=[]), Tige(queue=[]), Tige(queue=[]), Tige(queue=[])])
[Triplet(operation=(0, 1), noeudFils=Noeud(tiges=[Tige(queue=[]), Tige(queue=[Cube(nom=1)]), Tige(queue=[]), Tige(queue=[])]), cout=None), Triplet(operation=(0, 2), noeudFils=Noeud(tiges=[Tige(queue=[]), Tige(queue=[]), Tige(queue=[Cube(nom=1)]), Tige(queue=[])]), cout=None), Triplet(operation=(0, 3), noeudFils=Noeud(tiges=[Tige(queue=[]), Tige(queue=[]), Tige(queue=[]), Tige(queue=[Cube(nom=1)])]), cout=None)]


### Tests unitaire pour les fonctions

###Test pour les heuristiques

#####Test pour nombreMalMis

In [ ]:
### Cas =
noeud.print_noeud()
new_noeud = noeud.noeud_cp()
new_noeud.print_noeud()
print(nombreMalMis(noeud,new_noeud))

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
0


In [ ]:
### Cas !=
noeud.print_noeud()
new_noeud.get(0).pop()
new_noeud.get(1).push(list_cubes[0])
new_noeud.print_noeud()
print(nombreMalMis(noeud,new_noeud))

2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    1    |    |  
------------------------
       0    1    2    3
1


In [ ]:
### 2 noeuds
noeud.get(0).push(list_cubes[1])
noeud.print_noeud()
new_noeud.get(1).pop()
new_noeud.get(1).push(list_cubes[1])
new_noeud.get(1).push(list_cubes[0])
new_noeud.print_noeud()
print(nombreMalMis(noeud,new_noeud))

2 |    |    |    |    |  
1 |    2    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    1    |    |  
0 |    |    2    |    |  
------------------------
       0    1    2    3
2


###Tests pour les autres fonctions

In [ ]:
#Test opPos():
noeud.print_noeud()
for trip in opPos(noeud):
  trip.noeudFils.print_noeud()

2 |    |    |    |    |  
1 |    2    |    |    |  
0 |    1    |    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    2    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    2    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    |    |    2  
------------------------
       0    1    2    3


In [ ]:
new_noeud.print_noeud()
for trip in opPos(new_noeud):
  trip.noeudFils.print_noeud()

2 |    |    |    |    |  
1 |    |    1    |    |  
0 |    |    2    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    1    2    |    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    2    1    |  
------------------------
       0    1    2    3
2 |    |    |    |    |  
1 |    |    |    |    |  
0 |    |    2    |    1  
------------------------
       0    1    2    3


In [ ]:
#Test pour isInside():
list_noeuds = []
print(isInside(list_noeuds,noeud))
new_noeud = noeud.noeud_cp()
list_noeuds.append(new_noeud)
print(isInside(list_noeuds,noeud))
print(isInside(list_noeuds,new_noeud))

False
True
True


### Tests fonctionnels

####Variables globales

In [ ]:
#Initialisation noeud de depart
depart = []
for i in range(10):
  depart.append(Noeud([]))
  depart[i].init_noeud()

depart[0].get(0).push(Cube(1))

depart[1].get(0).push(Cube(1))
depart[1].get(0).push(Cube(2))

depart[2].get(0).push(Cube(3))
depart[2].get(0).push(Cube(2))
depart[2].get(0).push(Cube(1))

depart[3].get(0).push(Cube(3))
depart[3].get(0).push(Cube(2))
depart[3].get(0).push(Cube(1))
depart[3].get(1).push(Cube(4))


depart[4].get(0).push(Cube(3))
depart[4].get(0).push(Cube(2))
depart[4].get(0).push(Cube(1))
depart[4].get(1).push(Cube(5))
depart[4].get(1).push(Cube(4))

depart[5].get(0).push(Cube(3))
depart[5].get(0).push(Cube(2))
depart[5].get(0).push(Cube(1))
depart[5].get(1).push(Cube(6))
depart[5].get(1).push(Cube(5))
depart[5].get(1).push(Cube(4))

depart[6].get(0).push(Cube(3))
depart[6].get(0).push(Cube(2))
depart[6].get(0).push(Cube(1))
depart[6].get(1).push(Cube(6))
depart[6].get(1).push(Cube(5))
depart[6].get(1).push(Cube(4))
depart[6].get(2).push(Cube(7))

depart[9].get(0).push(Cube(3))
depart[9].get(0).push(Cube(2))
depart[9].get(0).push(Cube(1))
depart[9].get(1).push(Cube(6))
depart[9].get(1).push(Cube(5))
depart[9].get(1).push(Cube(4))
depart[9].get(2).push(Cube(9))
depart[9].get(2).push(Cube(8))
depart[9].get(2).push(Cube(7))

In [ ]:
#Initialisation noeud de but
but = []
for i in range(10):
  but.append(Noeud([]))
  but[i].init_noeud()

but[0].get(1).push(Cube(1))

but[1].get(0).push(Cube(2))
but[1].get(0).push(Cube(1))

but[2].get(0).push(Cube(1))
but[2].get(0).push(Cube(2))
but[2].get(0).push(Cube(3))

but[3].get(0).push(Cube(3))
but[3].get(0).push(Cube(2))
but[3].get(1).push(Cube(4))
but[3].get(2).push(Cube(1))

but[4].get(0).push(Cube(3))
but[4].get(0).push(Cube(2))
but[4].get(1).push(Cube(4))
but[4].get(2).push(Cube(5))
but[4].get(2).push(Cube(1))

but[5].get(0).push(Cube(3))
but[5].get(0).push(Cube(2))
but[5].get(1).push(Cube(6))
but[5].get(1).push(Cube(4))
but[5].get(2).push(Cube(5))
but[5].get(2).push(Cube(1))

but[6].get(0).push(Cube(3))
but[6].get(0).push(Cube(2))
but[6].get(0).push(Cube(7))
but[6].get(1).push(Cube(6))
but[6].get(1).push(Cube(4))
but[6].get(2).push(Cube(5))
but[6].get(2).push(Cube(1))

but[9].get(0).push(Cube(3))
but[9].get(0).push(Cube(2))
but[9].get(0).push(Cube(7))
but[9].get(1).push(Cube(6))
but[9].get(1).push(Cube(4))
but[9].get(1).push(Cube(8))
but[9].get(2).push(Cube(9))
but[9].get(2).push(Cube(5))
but[9].get(2).push(Cube(1))

####Test

In [ ]:
test = 9
depart[test].print_noeud()
but[test].print_noeud()
print("__________START___________")
sol,path,num_iter,num_noeuds_cree,num_noeuds_developpe = Ida_star(depart[test],but[test])
print("num_iter,num_noeuds_cree,num_noeuds_developpe:",(num_iter,num_noeuds_cree,num_noeuds_developpe))
#print_path(path)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    |    |  
1 |    2    1    9    8  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    |    |  
1 |    2    1    9    8  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    8    |  
1 |    2    1    9    |  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    8    |  
1 |    2    1    9    |  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    8    |  
1 |    2    1    9    |  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    8    |  
1 |    2    1    9    |  
0 |    3    6    4    7  
------------------------
       0    1    2    3
(3, 2)
2 |    5    |    8    |  
1 |    2    1    9    |  
0 |    3  